# Outline

1. Intro to topic modeling reading (~5 minutes)
2. Setup
3. Preview

![TM](./slides/tm-0.png)

![TM](./slides/tm-1.png)
![TM](./slides/tm-2.png)
![TM](./slides/tm-3.png)
![TM](./slides/tm-4.png)

# Setup

Run the cell below to import functions written by staff and functions from libraries. You will not need to worry about the details these, but if you are curious to see what they look like, checkout the file `tm_helpers.py`. If you want to learn more about how these functions or have questions, let us know! :)

In [1]:
import utils.tm_helpers as helpers

# general
from tqdm import tqdm
import os
import regex as re

# preprocess functions
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])


# topic modeling packages
import gensim
from gensim import models, corpora
from gensim.models.coherencemodel import CoherenceModel

# used to visualize the topic model
import pyLDAvis.gensim
import pyLDAvis

/usr/local/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/usr/local/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable


# Preview

By the end of this project, you will have experimented with preprocessing steps, and generate an interactive visual to explore the topics. Run the cell below to get a preview of the end goal.

In [23]:
helpers.show_model()

Generating visual, this will take a few moments...


PreparedData(topic_coordinates=              x        y  topics  cluster       Freq
topic                                               
2      0.081952  0.14081       1        1  35.040768
0     -0.177240 -0.00689       2        1  33.574291
1      0.095287 -0.13392       3        1  31.384935, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
453   Default  85.000000              cake  85.000000  30.0000  30.0000
6     Default  48.000000          computer  48.000000  29.0000  29.0000
1     Default  32.000000  computer_science  32.000000  28.0000  28.0000
1045  Default  22.000000              verb  22.000000  27.0000  27.0000
494   Default  25.000000           century  25.000000  26.0000  26.0000
825   Default  14.000000             vowel  14.000000  25.0000  25.0000
12    Default  20.000000            system  20.000000  24.0000  24.0000
1172  Default  13.000000       alternation  13.000000  23.0000  23.0000
0     Default  14.000000       computation  14.000000  22.0000  22.0000
821   Default  13.000000          syllable  13.000000  21.0000  21.0000
34    Default  13.000000       programming  13.000000  20.0000  20.0000
59    Default  13.000000       development  13.000000  19.0000  19.0000
3     Default  12.000000             study  12.000000  18.0000  18.0000
13    Default  12.000000            theory  12.000000  17.0000  17.0000
454   Default  13.000000             flour  13.000000  16.0000  16.0000
278   Default  37.000000              form  37.000000  15.0000  15.0000
8     Default  12.000000            design  12.000000  14.0000  14.0000
1140  Default  11.000000             class  11.000000  13.0000  13.0000
2     Default  11.000000       information  11.000000  12.0000  12.0000
23    Default  11.000000        discipline  11.000000  11.0000  11.0000
1089  Default  13.000000              noun  13.000000  10.0000  10.0000
1156  Default  10.000000         adjective  10.000000   9.0000   9.0000
24    Default  13.000000             field  13.000000   8.0000   8.0000
1141  Default  10.000000        declension  10.000000   7.0000   7.0000
11    Default  10.000000          software  10.000000   6.0000   6.0000
97    Default  10.000000           science  10.000000   5.0000   5.0000
15    Default  10.000000         computing  10.000000   4.0000   4.0000
457   Default  10.000000             sugar  10.000000   3.0000   3.0000
1139  Default   8.000000          singular   8.000000   2.0000   2.0000
1280  Default   8.000000        participle   8.000000   1.0000   1.0000
...       ...        ...               ...        ...      ...      ...
1059   Topic3   6.941285              loss   7.469290   1.0855  -4.9950
1047   Topic3   6.541629         diphthong   7.070148   1.0811  -5.0543
1180   Topic3   6.238444                je   6.765184   1.0778  -5.1017
1155   Topic3   6.237119              stem   6.765402   1.0775  -5.1020
1024   Topic3   5.293692             nasal   5.819977   1.0641  -5.2660
424    Topic3   4.731692            result   5.259983   1.0530  -5.3782
995    Topic3   4.358325                 t   4.885259   1.0447  -5.4604
993    Topic3   3.728317         consonant   4.255137   1.0267  -5.6165
1190   Topic3   3.654912       subjunctive   4.181459   1.0243  -5.6364
1142   Topic3   3.497585               -er   4.023951   1.0187  -5.6804
1089   Topic3  11.762541              noun  13.690912   1.0070  -4.4676
1133   Topic3   3.179449            neuter   3.709897   1.0045  -5.7758
816    Topic3   3.024989                 e   3.551349   0.9984  -5.8256
839    Topic3   3.024962                 l   3.551353   0.9984  -5.8256
952    Topic3   3.023741             music   3.551555   0.9980  -5.8260
757    Topic3   2.998004           cluster   3.525030   0.9969  -5.8345
1246   Topic3   2.794014            desjun   3.320170   0.9863  -5.9050
1205   Topic3   2.794011             value   3.320168   0.9863  -5.9050
1286   Topic3   2.79

# Data

## Load data

Run the cell below to load the data that we will use for topic modeling. The data is saved into a *list* data structure called `data`.

In [3]:
data = helpers.load_data()

**To get an idea of what `data` looks like, run the cell below to see what the first  looks like.**

In [26]:
print("Data length:", len(data), "First 10 items in data:\n")

# data is a list, data[:10] is the first ten items of that list
for i, item in enumerate(data[:10]):
    print("{}:".format(i), item)

Data length: 494 First 10 items in data:

0: Computer science is the study of computation and information.

1: Computer science deals with theory of computation, algorithms, computational problems and the design of computer systems hardware, software and applications.

2: Computer science addresses both human-made and natural information processes, such as communication, control, perception, learning and intelligence especially in human-made computing systems and machines.

3: According to Peter Denning, the fundamental question underlying computer science is, What can be automated?Its fields can be divided into theoretical and practical disciplines.

4: Computational complexity theory is highly abstract, while computer graphics and computational geometry emphasizes real-world applications.

5: Algorithmics is called the heart of computer science.

6: Programming language theory considers approaches to the description of computational processes, while software engineering involves the 

# Preprocessing Experiment 1

## Write the function `preprocess_line`.
1. At this point, *preprocessing* is the main piece to experiment with and observe your implementation decisions on topic modeling. Using at least one new thing that you learned in 1-Intro-to-NLP, write the function `preprocess_line` which takes in a string of text saved in the input variable `line`, so that the function returns a list of tokens called `preprocessed_line`. 
2. At the bottom of the cell, we call `preprocess_line` on `test_string` so you can quickly observe how your function is working. `test_string` is initially set to be the first string from `data`, but you can change `test_string` to another string from `data` or your own string to get a fuller idea of the effects of your function.
3. Take a second to record your decisions in the slide titled **Preprocessing Experiment 1**.


In [29]:
def preprocess_line(line):
    '''
    Fill in this function. Refer to 1-Intro-to-NLP for preprocessing ideas.
    '''
    preprocessed_line = []
    tokens = word_tokenize(line)
    
    # use spacy pipeline
    doc = nlp(" ".join(tokens))
    
    allowed_postags=['NOUN']
    
    # get pos_tags
    pos_tags = [token.pos_ for token in doc]
    
    # get_lemmas, also remove words that aren't in allowed pos tags, also remove stopwords
    lemmas = [token.lemma_ for token in doc if token.pos_ in allowed_postags and not token.is_stop]
    
    preprocessed_line = lemmas
    
    return preprocessed_line

test_string = data[0]
print(preprocess_line(test_string))

['computer', 'science', 'study', 'computation', 'information']


## Run `preprocess` on all of the data.
1. Run the cell below to `preprocess` all of the strings in `data` and save to `preprocessed_data` 
2. The cell will output a preview our `preprocessed_data`. How does is look different than our earlier preview? Do you have a bug or does it look how you want it to look? 
3. Take a second to record your observations in the slide deck, in the slide titled **Preprocessing Output Observations**.

In [28]:
def preprocess(data):
    preprocessed_data = []
    for line in tqdm(data):
        preprocessed_line = preprocess_line(line)
        preprocessed_data.append(preprocessed_line)
    return preprocessed_data
preprocessed_data = preprocess(data)

# data is a list, data[:10] is the first ten items of that list
for i, item in enumerate(preprocessed_data[:10]):
    print("{}:".format(i), item)

100%|██████████| 494/494 [00:01<00:00, 279.47it/s]

0: ['computer', 'science', 'study', 'computation', 'information']
1: ['computer', 'science', 'deal', 'theory', 'computation', 'algorithm', 'problem', 'design', 'computer', 'system', 'hardware', 'software', 'application']
2: ['computer', 'science', 'human', 'information', 'process', 'communication', 'control', 'perception', 'learning', 'intelligence', 'human', 'computing', 'system', 'machine']
3: ['question', 'computer', 'science', 'field', 'discipline']
4: ['complexity', 'theory', 'computer', 'graphic', 'geometry', 'world', 'application']
5: ['heart', 'computer', 'science']
6: ['programming', 'language', 'theory', 'approach', 'description', 'process', 'software', 'engineering', 'use', 'programming', 'language', 'system']
7: ['computer', 'architecture', 'computer', 'engineering', 'deal', 'construction', 'computer', 'component', 'computer', 'equipment']
8: ['computer', 'interaction', 'challenge', 'computer']
9: ['intelligence', 'goal', 'process', 'problem', 'decision', 'making', 'adaptat

## Final data preparation for gensim topic modeling

1. In this step, we use functions from the `gensim` module to create `dictionary` and `corpus` in the format that the topic modeling function requires.
2. `dictionary`: This is our *set of words* after our preprocessing, which we will use for creating different probability distributions. This *set of words* is often referred to as the *vocabulary* in NLP terminology.
3. `corpus`: Words are typically converted into a numerical representation before using them in a model. To demonstrate this, the cell will print each word for the first item in your preprocessed data next to its numerical encoding. Why is the word **information** represented as **(3, 1)**?

In [40]:
print("Creating dictionary and corpus instances for gensim...", end='')

dictionary = corpora.Dictionary(data_words_trigrams)
corpus = [dictionary.doc2bow(x) for x in data_words_trigrams]

print("complete.\n")

print(dictionary, '\n')


for original, encoded in zip(data_words_trigrams[0], corpus[0]):
    print("Original: {}".format(original), "--->  Encoded: {}".format(encoded))

Creating dictionary and corpus instances for gensim...complete.

Dictionary(1310 unique tokens: ['computation', 'computer_science', 'information', 'study', 'algorithm']...) 

Original: computer_science --->  Encoded: (0, 1)
Original: study --->  Encoded: (1, 1)
Original: computation --->  Encoded: (2, 1)
Original: information --->  Encoded: (3, 1)


# Create Topic Model


1. Run the cell below to create a topic model for our data, and the topic model will be saved in the variable `lda_model`. For now, do not worry about the parameters, we will experiment with those later.

In [43]:
'''
The following lines are used to setup the model's parameters. 
'''
UPDATE_EVERY = 10
CHUNKSIZE = 10
PASSES = 10
topic_model_settings = [{'num-topics':15, 'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':5,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':10,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}]
setting = topic_model_settings[0]
NUM_TOPICS = setting['num-topics']


'''
Here, we plug in our parameters and data into models.LdaModel to train the topic model. The topic model will be saved in the variable "lda_model".
'''
print("Training topic model of {} topics (this might take a moment)...".format(NUM_TOPICS), end='')
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state = setting['parameters']['random_state'], update_every = setting['parameters']['update_every'], chunksize = setting['parameters']['chunksize'], passes = setting['parameters']['passes'], alpha = setting['parameters']['alpha'], per_word_topics = setting['parameters']['per_word_topics'])
print("complete.")


Training topic model of 15 topics (this might take a moment)...complete.


# Qualitative Observations

We have created a topic model and saved it in `lda_model`. But what does this mean? Let's learn more as we perform some qualitative observations. Later, we will discuss two quantitative metrics, but a lot of times, topic models are decided from manual observation.

## Topic Terms
1. Run the cell below to observe the topic terms.  
2. By creating the topic model, we have created a probability distribution over our set of words (`dictionary` or the vocabulary) for 15 topics. We have created a helper function called `show_topic_terms` that will extract the ten words with the highest probabilities for fitting into each topic. 
3. The columns of `topic_terms` are the top ten words of each topic (Wn) and Wn's probability of belonging to each topic (Wn Pr). Each row is a topic.
4. What do you think of these topics? Can you make sense of clear topic divisions between the 15 topics, or, would you be able to give a name to each topic? How could you modify the `preprocess_line` function to possibly improve the topic model to get a clearer division between topics?
5. Before you make any changes or proceed, make notes of your observations in the slide **Topic Model Qualitative Observations**. You can include a screenshot of part of this table, paste the table values into the slides.

In [44]:
topic_terms = helpers.show_topic_terms(lda_model, NUM_TOPICS)
topic_terms

,W1,W1 Pr,W2,W2 Pr,W3,W3 Pr,W4,W4 Pr,W5,W5 Pr,W6,W6 Pr,W7,W7 Pr,W8,W8 Pr,W9,W9 Pr,W10,W10 Pr
Topic,,,,,,,,,,,,,,,,,,,,
0,object,0.045,computer,0.043,network,0.027,student,0.024,cryptography,0.022,system,0.020,database,0.019,procedure,0.017,feature,0.015,computation,0.014
1,form,0.103,verb,0.067,noun,0.039,case,0.036,flour,0.032,class,0.032,adjective,0.029,declension,0.029,sugar,0.025,singular,0.024
2,vowel,0.095,romance,0.077,nasal,0.034,paradigms,0.029,work,0.028,distinction,0.021,consonant,0.020,e,0.019,diphthong,0.014,style,0.011
3,computer,0.096,system,0.040,computer_science,0.039,programming,0.030,paradigm,0.029,engineering,0.029,computation,0.021,study,0.019,software_engineering,0.016,order,0.014
4,difference,0.040,group,0.026,note,0.022,system,0.020,devoicing,0.015,country,0.014,fricative,0.012,affricate,0.012,resource,0.012,domain,0.012
5,century,0.125,period,0.034,way,0.029,literature,0.026,diphthong,0.025,part,0.024,pronunciation,0.023,chanson,0.021,spelling,0.020,variety,0.018
6,language,0.091,computer_science,0.048,langue,0.035,dialect,0.030,time,0.027,area,0.026,type,0.023,discipline,0.020,datum,0.019,field,0.018
7,purpose,0.052,texture,0.027,substitute,0.017,field,0.010,cornstarch,0.009,tablespoon,0.009,cup,0.009,percentage,0.009,transmission,0.007,method,0.007
8,alternation,0.076,development,0.073,loss,0.040,je,0.036,subjunctive,0.021,li,0.017,cluster,0.017,desjun,0.015,lave,0.015,leve,0.015


## Topic Model Visualization

1. Run the cell below to generate an interactive visual to explore the topic model.
2. Add observations to the **Topic Model Qualitative Observations** slide.

In [45]:
helpers.show_model(lda_model, corpus, dictionary)

Generating visual, this will take a few moments...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.191655  0.310593       1        1  22.216148
1     -0.235287 -0.192514       2        1  14.615221
6      0.163535  0.050833       3        1  13.952335
3      0.155900  0.027422       4        1  10.102814
5     -0.025241 -0.026007       5        1   7.730519
2     -0.030645 -0.033952       6        1   5.881965
8     -0.035121 -0.044936       7        1   5.604486
12     0.072016 -0.013091       8        1   4.715309
9      0.039083 -0.001736       9        1   3.710989
14     0.028012 -0.008567      10        1   3.139445
0      0.046317 -0.008063      11        1   2.610370
4      0.025372 -0.010709      12        1   2.309147
13     0.010816 -0.013884      13        1   1.499629
10    -0.014997 -0.018377      14        1   1.078757
7     -0.008106 -0.017011      15        1   0.832875, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
454   Default  145.000000              cake  145.000000  30.0000  30.0000
279   Default   49.000000              form   49.000000  29.0000  29.0000
6     Default   53.000000          computer   53.000000  28.0000  28.0000
495   Default   32.000000           century   32.000000  27.0000  27.0000
33    Default   53.000000          language   53.000000  26.0000  26.0000
1046  Default   32.000000              verb   32.000000  25.0000  25.0000
826   Default   18.000000             vowel   18.000000  24.0000  24.0000
201   Default   34.000000              word   34.000000  23.0000  23.0000
731   Default   15.000000           romance   15.000000  22.0000  22.0000
1     Default   38.000000  computer_science   38.000000  21.0000  21.0000
1173  Default   14.000000       alternation   14.000000  20.0000  20.0000
59    Default   14.000000       development   14.000000  19.0000  19.0000
209   Default   28.000000           example   28.000000  18.0000  18.0000
12    Default   22.000000            system   22.000000  17.0000  17.0000
1090  Default   19.000000              noun   19.000000  16.0000  16.0000
822   Default   25.000000          syllable   25.000000  15.0000  15.0000
2     Default   11.000000       information   11.000000  14.0000  14.0000
393   Default   17.000000              case   17.000000  13.0000  13.0000
11    Default    8.000000          software    8.000000  12.0000  12.0000
696   Default   16.000000            langue   16.000000  11.0000  11.0000
3     Default   14.000000             study   14.000000  10.0000  10.0000
455   Default   15.000000             flour   15.000000   9.0000   9.0000
0     Default   15.000000       computation   15.000000   8.0000   8.0000
1141  Default   15.000000             class   15.000000   7.0000   7.0000
695   Default   14.000000           dialect   14.000000   6.0000   6.0000
13    Default   13.000000            theory   13.000000   5.0000   5.0000
1157  Default   14.000000         adjective   14.000000   4.0000   4.0000
8     Default   11.000000            design   11.000000   3.0000   3.0000
1142  Default   14.000000        declension   14.000000   2.0000   2.0000
34    Default   17.000000       programming   17.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
599   Topic15    0.018205             moist    0.741273   1.0814  -7.3093
369   Topic15    0.017119        automation    0.745598   1.0140  -7.3708
370   Topic15    0.017119      intervention    0.745598   1.0140  -7.3708
371   Topic15    0.017119        monitoring    0.745598   1.0140  -7.3708
1301  Topic15    0.016851        manchester    0.737589   1.0091  -7.3866
1302  Topic15    0.016851        phonétique    0.737589   1.0091  -7.3866
1300  Topic15    0.016851             maint    0.737589   1.0091  -7.3866
1299  Topic15    0.016851               cs1    0.737589   1.0091  -7.3866
1303 

# Quantitative Observations

1. We will compute *perplexity* and *coherence,* which are two quantitative metrics for evaluating topic models. Better scores do not necessarily mean better models in topic modeling. 
2. Add the scores to the **Topic Model 1 Quantitative Observations** slide.


## Perplexity

"In information theory, perplexity is a measurement of how well a probability distribution or probability model predicts a sample ([Wikipedia](https://en.wikipedia.org/wiki/Perplexity))."

Intuition: the dictionary definition of perplexed is "Completely baffled; very puzzled [Dictionary Reference](https://www.lexico.com/en/definition/perplexed)." The more perplexed you are, the more puzzled you are. Use this to remember the intuition behind the *perplexity* metric. We aim for a model that less perplexed by new data, so a smaller perplexity is usually the goal.


In [47]:
'''
Measure quality of topic models with perplexity
'''
print("Measuring model perplexity...",end="")
ppl = lda_model.log_perplexity(corpus)
print('complete. Perplexity:', ppl, '\n')  # a measure of how good the model is. lower the better.

Measuring model perplexity...complete. Perplexity: -9.470029050649728 



## Coherence

The coherence metric captures the "degree of semantic similarity between high scoring words in the topic ([Towards Data Science](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0))."

The idea behind the *coherence* metric aligns with your experience observing your topic terms. Did you observe a very clear theme in each topic or was there a lot of overlapping themes between the topics? Clear themes are analogous to higher coherence scores, whereas overlapping themes are analogous to distinct topics.

In [48]:
'''
Measure quality of topic models with coherence
'''
print("Measuring model coherence...",end="")
coherence_model_lda = CoherenceModel(model=lda_model, texts = data_words_trigrams[-10000:], corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('complete. Coherence Score:', coherence_lda, '\n') 

Measuring model coherence...complete. Coherence Score: 0.42054033797171636 



# Preprocessing Experiment 2

## Write the function `preprocess_line`.
1. At this point, *preprocessing* is the main piece to experiment with and observe your implementation decisions on topic modeling. Using at least one new thing that you learned in 1-Intro-to-NLP, write the function `preprocess_line` which takes in a string of text saved in the input variable `line`, so that the function returns a list of tokens called `preprocessed_line`. 
2. At the bottom of the cell, we call `preprocess_line` on `test_string` so you can quickly observe how your function is working. `test_string` is initially set to be the first string from `data`, but you can change `test_string` to another string from `data` or your own string to get a fuller idea of the effects of your function.
3. Take a second to record your decisions in the slide titled **Preprocessing Experiment 1**.


In [ ]:
def preprocess_line(line):
    '''
    Fill in this function. Refer to 1-Intro-to-NLP for preprocessing ideas.
    '''
    preprocessed_line = []
    tokens = word_tokenize(line)
    
    # use spacy pipeline
    doc = nlp(" ".join(tokens))
    
    allowed_postags=['NOUN']
    
    # get pos_tags
    pos_tags = [token.pos_ for token in doc]
    
    # get_lemmas, also remove words that aren't in allowed pos tags, also remove stopwords
    lemmas = [token.lemma_ for token in doc if token.pos_ in allowed_postags and not token.is_stop]
    
    preprocessed_line = lemmas
    
    return preprocessed_line

test_string = data[0]
print(preprocess_line(test_string))

# Learn about Topic Model Parameters

In [ ]:
UPDATE_EVERY = 10
CHUNKSIZE = 10
PASSES = 10
topic_model_settings = [{'num-topics':15, 'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':5,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':10,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}]
setting = topic_model_settings[0]
NUM_TOPICS = setting['num-topics']

**We've already come up with a few different parameter settings, the main difference being the number of topics we are targetting. Let's start with the first setting, which will create 15 topics.**

In [14]:
setting = topic_model_settings[0]
NUM_TOPICS = setting['num-topics']
NUM_TOPICS = 3

## 2. Create a topic model

**Now we create a topic model for our text using functions from gensim! This will take a few minutes, so take this time to review some content. You can check out [this resource](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) on topic modeling as well, which will show several of the steps we have already covered in a bit more detail, and give you a preview of what we will do next!**

In [15]:
print("Training topic model (this will take a moment)...", end='')
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state = setting['parameters']['random_state'], update_every = setting['parameters']['update_every'], chunksize = setting['parameters']['chunksize'], passes = setting['parameters']['passes'], alpha = setting['parameters']['alpha'], per_word_topics = setting['parameters']['per_word_topics'])
print("complete.")


Training topic model (this will take a moment)...complete.


## 3. Observe words associated with the topic

The columns of `topic_terms` are the top ten words of each topic (Wn) and Wn's probability of belonging to each topic (Wn Pr). 

In [16]:
topic_terms = helpers.show_topic_terms(lda_model, NUM_TOPICS)
topic_terms

,W1,W1 Pr,W2,W2 Pr,W3,W3 Pr,W4,W4 Pr,W5,W5 Pr,W6,W6 Pr,W7,W7 Pr,W8,W8 Pr,W9,W9 Pr,W10,W10 Pr
Topic,,,,,,,,,,,,,,,,,,,,
0,cake,0.038,computer,0.017,century,0.014,computer_science,0.011,information,0.010,programming,0.009,language,0.009,word,0.009,study,0.009,example,0.008
1,cake,0.031,computer,0.020,form,0.014,computer_science,0.013,verb,0.013,language,0.009,vowel,0.008,system,0.008,romance,0.007,alternation,0.007
2,language,0.015,noun,0.014,declension,0.013,class,0.013,adjective,0.011,form,0.011,history,0.008,case,0.008,term,0.008,computer,0.008


## 4. Evaluate quantitatively

**We will use two measures to evaluate the model: perplexity and coherence**

Todo: I'll expand on this a little bit to explain what these metrics mean

In [17]:
'''
6. Measure quality of topic models with perplexity
'''
print("Measuring model perplexity...",end="")
ppl = lda_model.log_perplexity(corpus)
print('complete. Perplexity:', ppl, '\n')  # a measure of how good the model is. lower the better.

'''
7. Measure quality of topic models with coherence
'''
print("Measuring model coherence...",end="")
coherence_model_lda = CoherenceModel(model=lda_model, texts = data_words_trigrams[-10000:], corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('complete. Coherence Score:', coherence_lda, '\n') 

Measuring model perplexity...complete. Perplexity: -6.982638798070078 

Measuring model coherence...complete. Coherence Score: 0.524049569197438 



## 5. Assign text lines to a topic

In [18]:
from collections import defaultdict
import operator
'''
9. Save topic assignments for unique texts in a datastructure
'''
print("Making document topic assignments...")
text2distro = {}
for x in range(len(corpus)):
    topicdistribution = lda_model[corpus[x]]     # a list of tuples, e.g., [(8, 0.14625458), (10, 0.79183161)]
    topicarray = [0]*NUM_TOPICS

    for (topicid,topicprc) in topicdistribution:
        topicarray[topicid] = topicprc
    try:
        text2distro[' '.join(data_words_trigrams[-10000:][x])] = topicarray
    except:
        print('x:', x, "len(data_words_trigrams[-10000:]):", len(data_words_trigrams[-10000:]))

top_topics = defaultdict(lambda:0)
second_top_topics = defaultdict(lambda:0)
text2scores = defaultdict(lambda:0)

for text in text2distro:
    if len(text) > 1:
        distro = text2distro[text]
        idx2score = {i:score for i, score in enumerate(distro)}
        scores_sorted = sorted(idx2score.items(), key=operator.itemgetter(1), reverse=True)
        top_topic = scores_sorted[0][0]
        top_topics[top_topic] += 1
        second_top_topics[scores_sorted[1][0]] += 1
        text2scores[text] = scores_sorted

# print("Top topic distribution:\n", top_topics)
# print("\n\nSecond top topic distribution:\n", second_top_topics)

print("Topic #", '\t', "% docs assigned")
total = sum(top_topics.values())
for i in range(NUM_TOPICS):
    print(i, '\t', "{:.2%}".format(top_topics[i]/total))
print('complete.\n')

Making document topic assignments...
Topic # 	 % docs assigned
0 	 33.19%
1 	 49.12%
2 	 17.70%
complete.



## 5. Visualize the topics

**It will take a few moments to load up the visual**

In [19]:
helpers.show_model(lda_model, corpus, dictionary)

Generating visual, this will take a few moments...


/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.018805 -0.095574       1        1  47.888680
0     -0.096013  0.032224       2        1  35.818119
2      0.077209  0.063350       3        1  16.293200, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
454   Default  94.000000              cake  94.000000  30.0000  30.0000
1090  Default  10.000000              noun  10.000000  29.0000  29.0000
1142  Default   7.000000        declension   7.000000  28.0000  28.0000
1141  Default   8.000000             class   8.000000  27.0000  27.0000
1157  Default   6.000000         adjective   6.000000  26.0000  26.0000
1046  Default  20.000000              verb  20.000000  25.0000  25.0000
34    Default  13.000000       programming  13.000000  24.0000  24.0000
52    Default   5.000000           history   5.000000  23.0000  23.0000
33    Default  33.000000          language  33.000000  22.0000  22.0000
459   Default   9.000000             bread   9.000000  21.0000  21.0000
2     Default  13.000000       information  13.000000  20.0000  20.0000
92    Default  10.000000              term  10.000000  19.0000  19.0000
495   Default  23.000000           century  23.000000  18.0000  18.0000
1112  Default   4.000000      stress_shift   4.000000  17.0000  17.0000
129   Default   4.000000        transistor   4.000000  16.0000  16.0000
279   Default  33.000000              form  33.000000  15.0000  15.0000
393   Default  11.000000              case  11.000000  14.0000  14.0000
10    Default   7.000000           problem   7.000000  13.0000  13.0000
826   Default  14.000000             vowel  14.000000  12.0000  12.0000
3     Default  14.000000             study  14.000000  11.0000  11.0000
113   Default  11.000000              area  11.000000  10.0000  10.0000
1140  Default   7.000000          singular   7.000000   9.0000   9.0000
22    Default   7.000000           process   7.000000   8.0000   8.0000
1173  Default  12.000000       alternation  12.000000   7.0000   7.0000
430   Default   2.000000           student   2.000000   6.0000   6.0000
209   Default  14.000000           example  14.000000   5.0000   5.0000
112   Default   4.000000        department   4.000000   4.0000   4.0000
60    Default   5.000000        calculator   5.000000   3.0000   3.0000
234   Default   5.000000         paradigms   5.000000   2.0000   2.0000
507   Default   2.000000              milk   2.000000   1.0000   1.0000
...       ...        ...               ...        ...      ...      ...
1039   Topic3   0.734311             count   1.237634   1.2924  -6.5857
1145   Topic3   0.734311         masculine   1.237635   1.2924  -6.5857
1104   Topic3   0.734309               OFr   1.237633   1.2924  -6.5857
1036   Topic3   0.734311            bounty   1.237635   1.2924  -6.5857
1035   Topic3   0.734304             bonté   1.237636   1.2924  -6.5857
703    Topic3   1.100757              half   1.862481   1.2885  -6.1809
1143   Topic3   1.755749               -er   3.149309   1.2301  -5.7140
112    Topic3   2.368908        department   4.406565   1.1938  -5.4145
1134   Topic3   1.689885            neuter   3.178638   1.1826  -5.7522
10     Topic3   3.445415           problem   7.853913   0.9905  -5.0399
92     Topic3   4.294747              term  10.393433   0.9306  -4.8195
60     Topic3   2.374822        calculator   5.182805   1.0340  -5.4120
727    Topic3   1.834917            change   3.800999   1.0862  -5.6699
22     Topic3   2.932270           process   7.147738   0.9234  -5.2011
393    Topic3   4.314905              case  11.977467   0.7935  -4.8148
33     Topic3   7.894189          language  33.399097   0.3720  -4.2108
279    Topic3   6.020164              form  33.754959   0.0904  -4.4818
1140   Topic3   2.641298          singular   7.458184   0.7764  -5.3056
0      Topic3  

## Join bigrams and trigrams
1. Next we will train a bigram model by using functions implemented by others (gensim) for us to use!
2. Now we apply our bigram model to our data to join unigrams into bigrams where appropriate. To understand what the changes look like, the `show_ngrams` function will output a few examples of lines that were changed by this process.

In [8]:
bigram_model, bigram_phrases = helpers.train_bigram_model(preprocessed_data)
data_words_bigrams = helpers.make_bigrams(preprocessed_data, bigram_model)

helpers.show_ngrams(preprocessed_data, data_words_bigrams)

3. Now we will train a trigram model and apply it to our data to join unigrams and bigrams where appropriate. 
5. To understand what the changes look like, the `show_ngrams` function will output a few examples of lines that were changed by this process.

In [30]:
trigram_model = helpers.train_trigram_model(preprocessed_data, bigram_phrases)

data_words_trigrams = helpers.make_trigrams(data_words_bigrams, bigram_model, trigram_model)
helpers.show_ngrams(data_words_bigrams, data_words_trigrams)

100%|██████████| 494/494 [00:00<00:00, 20309.81it/s]


,Before,After


# References

1. https://dl.acm.org/doi/fullHtml/10.1145/2133806.2133826
2. https://nlpforhackers.io/topic-modeling/
3. https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/  -- for more analyses of topic models